<a href="https://colab.research.google.com/github/sandroormeno/TensorFlow-LITE-ESP32-ARDUINO/blob/master/tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install tensorflow==2.1.0

     |████████████████████████████████| 421.8MB 36kB/s 
     |████████████████████████████████| 450kB 35.6MB/s 
  Created wheel for gast: filename=gast-0.2.2-cp36-none-any.whl size=7540 sha256=039ff8f31ba1294f41aea85bec6746b37d83f6b6f01376a3d218a4b5cdfa0a25
  Stored in directory: /root/.cache/pip/wheels/5c/2e/7e/a1d4d4fcebe6c381f378ce7743a3ced3699feb89bcfbdadadd
Successfully built gast
  Found existing installation: tensorflow-estimator 2.2.0rc0
    Uninstalling tensorflow-estimator-2.2.0rc0:
      Successfully uninstalled tensorflow-estimator-2.2.0rc0
  Found existing installation: gast 0.3.3
    Uninstalling gast-0.3.3:
      Successfully uninstalled gast-0.3.3
  Found existing installation: tensorflow 2.2.0rc1
    Uninstalling tensorflow-2.2.0rc1:
      Successfully uninstalled tensorflow-2.2.0rc1


In [0]:
%tensorflow_version 2.x

In [2]:
from __future__ import absolute_import, division, print_function, unicode_literals
import pandas
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers
from keras.utils import to_categorical, np_utils
from sklearn.model_selection import train_test_split
from keras import backend as K

Using TensorFlow backend.


In [3]:
print("TensorFlow version: {}".format(tf.__version__))

TensorFlow version: 2.1.0


In [4]:
url = "https://raw.githubusercontent.com/sandroormeno/Tensorflow-basic-examples/master/datos/iris.csv"
datosIris = pandas.read_csv(url)
datosIris.head()

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa


In [0]:
datos=datosIris.values[:,:4]
etiquetas=datosIris.values[:,4]

In [0]:
X_train,X_test,y_train,y_test=train_test_split(datos,etiquetas,train_size=0.8,random_state=23)

In [0]:
def one_hot(arr):
    uniques, ids = np.unique(arr, return_inverse=True)
    return np_utils.to_categorical(ids, len(uniques))
y_train_ohe=one_hot(y_train)
y_test_ohe=one_hot(y_test)

In [0]:
# los datos de entrada dan un error, al no poder convertir Float en tensores.
# soluciòn aquì : https://blog.csdn.net/weixin_44473755/article/details/102939703
X_train_1 = K.cast_to_floatx (X_train)
X_test_1 = K.cast_to_floatx (X_test)

In [11]:
print (X_train_1[20] )

[7.4 2.8 6.1 1.9]


In [0]:
# El modelo
model= tf.keras.Sequential()

model.add(layers.Dense(16, activation='relu', input_dim=4))
model.add(layers.Dense(16, activation='relu'))
model.add(layers.Dense(3, activation='sigmoid'))

# Configulación del entrenamiento
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [13]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 16)                80        
_________________________________________________________________
dense_1 (Dense)              (None, 16)                272       
_________________________________________________________________
dense_2 (Dense)              (None, 3)                 51        
Total params: 403
Trainable params: 403
Non-trainable params: 0
_________________________________________________________________


In [14]:
model.fit(X_train_1,y_train_ohe, epochs=100,batch_size=1,verbose=1)

Train on 120 samples
Epoch 1/100
120/120 [==============================] - 0s 4ms/sample - loss: 0.8802 - accuracy: 0.5083
Epoch 2/100
120/120 [==============================] - 0s 2ms/sample - loss: 0.6837 - accuracy: 0.6833
Epoch 3/100
120/120 [==============================] - 0s 1ms/sample - loss: 0.5258 - accuracy: 0.8250
Epoch 4/100
120/120 [==============================] - 0s 2ms/sample - loss: 0.4314 - accuracy: 0.6833
Epoch 5/100
120/120 [==============================] - 0s 1ms/sample - loss: 0.3949 - accuracy: 0.7833
Epoch 6/100
120/120 [==============================] - 0s 1ms/sample - loss: 0.3074 - accuracy: 0.9500
Epoch 7/100
120/120 [==============================] - 0s 1ms/sample - loss: 0.2337 - accuracy: 0.9250
Epoch 8/100
120/120 [==============================] - 0s 1ms/sample - loss: 0.2083 - accuracy: 0.9417
Epoch 9/100
120/120 [==============================] - 0s 2ms/sample - loss: 0.1979 - accuracy: 0.9333
Epoch 10/100
120/120 [==============================

In [15]:
loss, accuracy = model.evaluate(X_test_1, y_test_ohe, verbose=0)
print("Test accuracy : " + str(accuracy))

Test accuracy : 0.96666664


In [16]:
# Stay here
# Convert the model to the TensorFlow Lite format without quantization
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.OPTIMIZE_FOR_SIZE]
tflite_model = converter.convert()

# Save the model to disk
open("exercice_model.tflite", "wb").write(tflite_model)

3152

In [18]:
X_test = X_test.astype('float32')

# Instantiate an interpreter for each model
exercice_model = tf.lite.Interpreter('exercice_model.tflite')

# Allocate memory for each model
exercice_model.allocate_tensors()

# Get the input and output tensors so we can feed in values and get the results
input_details = exercice_model.get_input_details()
output_details = exercice_model.get_output_details()

test = 15

input_data = X_test_1[test].reshape((1,4))
exercice_model.set_tensor(input_details[0]['index'],input_data )

exercice_model.invoke()
  
output_data = exercice_model.get_tensor(output_details[0]['index'])
    
  
print(output_data)  
print(y_test[test])

[[1.7133393e-08 1.9757006e-02 7.4578724e-03]]
Iris-virginica


In [19]:
import os
basic_model_size = os.path.getsize("exercice_model.tflite")
print("Basic model is %d bytes" % basic_model_size)

Basic model is 3152 bytes


In [21]:
# Install xxd if it is not available
!sudo apt-get update
!sudo apt-get install xxd


Ign:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:2 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:3 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Get:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release [564 B]
Get:7 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran35/ InRelease [3,622 B]
Get:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release.gpg [833 B]
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:10 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:11 http://ppa.launchpad.net/marutter/c2d4u3.5/ubuntu bionic InRelease [15.4 kB]
Get:13 http://archive.ubuntu.com/ubun

In [22]:
# Save the file as a C source file
!xxd -i exercice_model.tflite > exercice_model.cc
# Print the source file
!cat exercice_model.cc

unsigned char exercice_model_tflite[] = {
  0x1c, 0x00, 0x00, 0x00, 0x54, 0x46, 0x4c, 0x33, 0x00, 0x00, 0x12, 0x00,
  0x1c, 0x00, 0x04, 0x00, 0x08, 0x00, 0x0c, 0x00, 0x10, 0x00, 0x14, 0x00,
  0x00, 0x00, 0x18, 0x00, 0x12, 0x00, 0x00, 0x00, 0x03, 0x00, 0x00, 0x00,
  0xfc, 0x0b, 0x00, 0x00, 0x50, 0x07, 0x00, 0x00, 0x38, 0x07, 0x00, 0x00,
  0x34, 0x00, 0x00, 0x00, 0x04, 0x00, 0x00, 0x00, 0x01, 0x00, 0x00, 0x00,
  0x04, 0x00, 0x00, 0x00, 0xa8, 0xf4, 0xff, 0xff, 0x08, 0x00, 0x00, 0x00,
  0x0c, 0x00, 0x00, 0x00, 0x13, 0x00, 0x00, 0x00, 0x6d, 0x69, 0x6e, 0x5f,
  0x72, 0x75, 0x6e, 0x74, 0x69, 0x6d, 0x65, 0x5f, 0x76, 0x65, 0x72, 0x73,
  0x69, 0x6f, 0x6e, 0x00, 0x0d, 0x00, 0x00, 0x00, 0xf8, 0x06, 0x00, 0x00,
  0x28, 0x06, 0x00, 0x00, 0x04, 0x06, 0x00, 0x00, 0xb4, 0x05, 0x00, 0x00,
  0xa4, 0x01, 0x00, 0x00, 0x54, 0x01, 0x00, 0x00, 0x4c, 0x01, 0x00, 0x00,
  0x44, 0x01, 0x00, 0x00, 0x3c, 0x01, 0x00, 0x00, 0x2c, 0x00, 0x00, 0x00,
  0x24, 0x00, 0x00, 0x00, 0x1c, 0x00, 0x00, 0x00, 0x04, 0x00, 0x00, 0x

In [23]:
for i in range(len(X_test_1)):
  salida = str(X_test_1[i]) + str(y_test_ohe[i])
  print(salida)

[6.9 3.1 5.4 2.1][0. 0. 1.]
[7.2 3.2 6.  1.8][0. 0. 1.]
[5.8 2.7 4.1 1. ][0. 1. 0.]
[4.6 3.1 1.5 0.2][1. 0. 0.]
[5.7 2.5 5.  2. ][0. 0. 1.]
[6.9 3.1 4.9 1.5][0. 1. 0.]
[5.7 3.8 1.7 0.3][1. 0. 0.]
[6.4 3.1 5.5 1.8][0. 0. 1.]
[5.4 3.4 1.7 0.2][1. 0. 0.]
[5.6 2.9 3.6 1.3][0. 1. 0.]
[6.1 2.8 4.7 1.2][0. 1. 0.]
[4.8 3.1 1.6 0.2][1. 0. 0.]
[7.1 3.  5.9 2.1][0. 0. 1.]
[5.1 3.8 1.5 0.3][1. 0. 0.]
[5.4 3.7 1.5 0.2][1. 0. 0.]
[6.3 2.8 5.1 1.5][0. 0. 1.]
[6.4 3.2 4.5 1.5][0. 1. 0.]
[6.1 2.9 4.7 1.4][0. 1. 0.]
[7.7 3.  6.1 2.3][0. 0. 1.]
[5.2 3.4 1.4 0.2][1. 0. 0.]
[6.4 3.2 5.3 2.3][0. 0. 1.]
[4.8 3.  1.4 0.3][1. 0. 0.]
[5.1 3.8 1.6 0.2][1. 0. 0.]
[4.7 3.2 1.3 0.2][1. 0. 0.]
[6.7 3.1 5.6 2.4][0. 0. 1.]
[5.1 3.5 1.4 0.3][1. 0. 0.]
[5.1 3.8 1.9 0.4][1. 0. 0.]
[6.2 2.8 4.8 1.8][0. 0. 1.]
[6.6 3.  4.4 1.4][0. 1. 0.]
[5.6 2.5 3.9 1.1][0. 1. 0.]


In [24]:
print(X_test_1.shape)

(30, 4)


In [26]:
print(y_test_ohe)

[[0. 0. 1.]
 [0. 0. 1.]
 [0. 1. 0.]
 [1. 0. 0.]
 [0. 0. 1.]
 [0. 1. 0.]
 [1. 0. 0.]
 [0. 0. 1.]
 [1. 0. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [1. 0. 0.]
 [0. 0. 1.]
 [1. 0. 0.]
 [1. 0. 0.]
 [0. 0. 1.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 0. 1.]
 [1. 0. 0.]
 [0. 0. 1.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [0. 0. 1.]
 [1. 0. 0.]
 [1. 0. 0.]
 [0. 0. 1.]
 [0. 1. 0.]
 [0. 1. 0.]]
